In [1]:
## checking and installing papermill in lab env
# uncomented for testing - gas
# !pip list --isolated
!pip install papermill

In [2]:
import datetime
import json
import logging
import os
import papermill as pm

In [4]:
# Notebook Details

# Description of Notebooks purpose
description="Generate OpenAPI service definition of all Service notebooks in this directory"

# Filename where results will be written 
output = "output.json"

# Input JSON structure
input={ "service-url": "http://test-app-datamon-test.apps.65af81ab8957d90010952146.cloud.techzone.ibm.com"}

# Results JSON structure
results = {}

In [5]:
openAPI = {
	"swagger": "2.0",
	"info": {
		"title": "New ESG Services",
		"version": "1.0.43",
		"description": "High value services to make your life easier using the most comprehensive ESG data sets in the world",
		"x-ibm-name": "new-esg-services"
	},
	"basePath": "/api",
	"host": "$(catalog.host)",
	"schemes": [
		"https"
	],
	"x-ibm-configuration": {
		"cors": {
			"enabled": True
		},
		"properties": {
			"target-url": {
				"description": "The URL of the Code Engine service",
				"encoded": False,
				"value": input['service-url']
			}
		},
		"phase": "realized",
		"enforced": True,
		"gateway": "datapower-api-gateway",
		"assembly": {
			"execute": [
				{
					"invoke": {
						"title": "invoke",
						"version": "2.0.0",
						"verb": "keep",
						"target-url": "$(target-url)$(request.path)",
						"follow-redirects": False,
						"timeout": 60,
						"parameter-control": {
							"type": "blocklist"
						},
						"header-control": {
							"type": "blocklist",
							"values": [
								"^X-IBM-Client-Id$"
							]
						},
						"inject-proxy-headers": True
					}
				}
			],
			"finally": []
		},
		"type": "rest",
		"testable": True,
		"activity-log": {
			"enabled": True,
			"success-content": "activity",
			"error-content": "payload"
		}
	},
	"securityDefinitions": {
		"clientID": {
			"type": "apiKey",
			"in": "header",
			"name": "X-IBM-Client-Id"
		}
	},
	"security": [
		{
			"clientID": []
		}
	]
}

In [6]:
path = {
    'post': {
        'summary': 'Returns an ESG analysis of an asset portfolio', # needs substitution
        'consumes': ['application/json'],
        'parameters': [
            {'in': 'body',
              'name': 'ESGPortfolioService', # needs substiution
              'description': 'Returns an ESG analysis of an asset portfolio', # needs substiution
              'schema': {
                  '$ref': '#/definitions/ESGPortfolioService' # needs substiution
                  }
                }
            ],
        'responses': {
            '200': {
                'description': 'success',
                          'schema': {
                                "type": "object",
                                 "example": "value"
                                  },

                }
            }
        }
    }


In [7]:
definition = {
			"type": "object",
			"example": {
            }
    }

In [8]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk("."):
    f.extend(filenames)
    break

In [9]:
filenames

['.dockerignore',
 'requirements.txt',
 'ESGBackendNotebook.ipynb',
 'README.md',
 'wsgi.py',
 'wrapper.py',
 'ESGFastTickerService-ibmcloud.ipynb.bak',
 'cra-custom-script.sh',
 'calculateAgeService.ipynb',
 'ESG-consolidated.csv',
 'openapi-generator.ipynb',
 'ESGFastTickerService.ipynb',
 '.gitignore',
 'Dockerfile',
 'output.json']

In [10]:
from copy import deepcopy

paths = {}
definitions = {}
    
for file in filenames:
    if file.endswith("Service.ipynb"):
        inspect = pm.inspect_notebook("./"+file)
        #print("==========INSPECT==========")
        #print(json.dumps(inspect, indent = 3))
        #print("=========INSPECT ENDS=========")
        apiName = file.strip(".ipynb")
        #print("apiName:",apiName)
        newPath = deepcopy(path)
        newPath['post']['summary'] = inspect['description']['default'].strip('\"')
        newPath['post']['parameters'][0]['name'] = apiName
        newPath['post']['parameters'][0]['description'] = inspect['description']['default'].strip('\"')
        newPath['post']['parameters'][0]['schema']['$ref'] = '#/definitions/' + apiName
        newPath['post']['responses']['200']['schema']['example'] = inspect['results']['default'].strip('\"') 
        #newPath['post']['requestBody']['content']['application/json']['examples'] = {"0" : { "value": inspect['input']['default'] }}
        #newPath['post']['responses']['200']['content']['application/json']['examples'] = {"0" : { "value": inspect['results']['default'] }}
        # print(newPath['post']['requestBody']['content']['application/json']['schema']['$ref'])
        #newPath['post']['requestBody']['content']['application/json']['schema']['$ref']= newPath['post']['requestBody']['content']['application/json']['schema']['$ref'] + apiName
        #print(newPath)
        #newPath['post']['servers'][0]['url'] = "https://datamon-toolchain.vlu1sm4dps1.eu-gb.codeengine.appdomain.cloud"
        # newPath['servers'][0]['url'] = "https://datamon-toolchain.vlu1sm4dps1.eu-gb.codeengine.appdomain.cloud"
        #print(json.dumps(newPath, indent=4))
        key = '/'+apiName
        paths[key] = newPath
        #newSchema = deepcopy(schema)
        #schemas[apiName] = newSchema
        newDefinition = deepcopy(definition)
        newDefinition['example'] = inspect['input']['default'].strip('\"') 
        definitions[apiName] = newDefinition


In [11]:
openAPI['paths'] = paths 
openAPI['definitions'] = definitions

In [12]:
jsonString = json.dumps(openAPI, indent=4)
print(jsonString)

{
    "swagger": "2.0",
    "info": {
        "title": "New ESG Services",
        "version": "1.0.23",
        "description": "High value services to make your life easier using the most comprehensive ESG data sets in the world",
        "x-ibm-name": "new-esg-services"
    },
    "basePath": "/api",
    "host": "$(catalog.host)",
    "schemes": [
        "https"
    ],
    "x-ibm-configuration": {
        "cors": {
            "enabled": true
        },
        "properties": {
            "target-url": {
                "description": "The URL of the Code Engine service",
                "encoded": false,
                "value": "http://test-app-datamon-test.apps.65af81ab8957d90010952146.cloud.techzone.ibm.com"
            }
        },
        "phase": "realized",
        "enforced": true,
        "gateway": "datapower-api-gateway",
        "assembly": {
            "execute": [
                {
                    "invoke": {
                        "title": "invoke",
            

In [14]:
# Serializing json
json_object = json.dumps(openAPI, indent=4)
 
# Writing to sample.json
with open(output, "w") as outfile:
    outfile.write(json_object)